In [39]:
import json
from tqdm import tqdm
import googlemaps
with open("data.json") as f:
    data = json.load(f)
print(len(data))

6023


In [30]:
data[2]

{'id': 2116846707,
 'date_published': '2009-10-06',
 'citation_count': 434,
 'estimated_citation_count': 1181,
 'title': 'Suspending Damage: A Letter to Communities',
 'authors': [{'DAuN': 'Eve Tuck',
   'AuId': 2165230311,
   'DAfN': 'State University of New York System',
   'AfId': 1327163397}],
 'fields': [{'DFN': 'Oppression', 'FId': 2776526686},
  {'DFN': 'Social exclusion', 'FId': 134659438},
  {'DFN': 'Accountability', 'FId': 2776007630},
  {'DFN': 'Educational research', 'FId': 36727532},
  {'DFN': 'Disadvantaged', 'FId': 2780623907},
  {'DFN': 'Environmental ethics', 'FId': 95124753},
  {'DFN': 'Theory of change', 'FId': 7340896},
  {'DFN': 'Law', 'FId': 199539241},
  {'DFN': 'Conflict resolution', 'FId': 21711469},
  {'DFN': 'Sociology', 'FId': 144024400},
  {'DFN': 'Perspective-taking', 'FId': 2780342009}],
 'references': [2071275339]}

In [31]:
paper_lookup = {}
for node in data:
    paper_lookup[node["id"]] = node

In [36]:
institutions = {}
for node in data:
    seen_inst_this_node = []
    for a in node.get("authors"):
        id = a.get("AfId")
        if id and id not in seen_inst_this_node:
            seen_inst_this_node.append(id)
            if id not in institutions:
                institutions[id] = {
                    "id": id,
                    "name": a.get("DAfN"),
                    "n_papers": 0,
                    "n_citations": 0,
                    "links": {}
                }
            institutions[id]["n_papers"] += 1
            institutions[id]["n_citations"] += node["citation_count"]
            for ref in node["references"]:
                other = paper_lookup[ref]
                for oa in other.get("authors"):
                    oid = oa.get("AfId")
                    if oid:
                        if oid not in institutions[id]["links"]:
                            institutions[id]["links"][oid] = 0
                        institutions[id]["links"][oid] += 1
institutions = list(institutions.values())
print(len(institutions))

966


In [48]:
gmaps = googlemaps.Client(key='INSERT_KEY_HERE')
for inst in tqdm(institutions):
    if "geo" not in inst:
        inst["geo"] = gmaps.geocode(inst["name"])

100%|██████████| 966/966 [09:40<00:00,  1.66it/s]


In [51]:
with open("institutions.json", "w") as f:
    json.dump(institutions, f)
!ls -lah institutions.json

-rw-r--r-- 1 nyou045 nyou045 1.3M Nov 26 15:16 institutions.json
